In [ ]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
traj = md.load('trajectory.dcd', top='topology.prmtop')

In [ ]:
prot = md.load_pdb('pdb_from_tleap.pdb')

In [ ]:
cas = [atom.index for atom in prot.topology.atoms if (atom.name == 'CA')]
residues= [str(res) for res in prot.topology.residues]

In [ ]:
for res in residues:
    if res.startswith('ACE'):
        residues.remove(res)

In [ ]:
for res in residues:
    if res.startswith('NHE'):
        residues.remove(res)

In [ ]:
# anchor points for Boresch restraints
idx = 0

for i in residues:
    if i == 'ILE11':
        c_place = idx
    elif i == 'LYS16':
        b_place = idx
    elif i == 'ASN18':
        a_place = idx
    elif i == 'ASN52':
        A_place = idx
    elif i == 'TRP57':
        B_place = idx
    elif i == 'ILE59':
        C_place = idx
    idx+=1

c_idx = cas[c_place]
b_idx = cas[b_place]
a_idx = cas[a_place]
A_idx = cas[A_place]
B_idx = cas[B_place]
C_idx = cas[C_place]

In [ ]:
def angle_calc(point1, point2, point3):
    vector1 = point1 - point2
    vector2 = point3 - point2

    dot_product = np.vdot(vector1, vector2)
    mag_vector1 = np.linalg.norm(vector1)
    mag_vector2 = np.linalg.norm(vector2)
    
    angle = np.degrees(np.arccos(dot_product/(mag_vector1*mag_vector2)))
    
    return angle


In [ ]:
def dihedral_calc(point0, point1, point2, point3):
    b0 = -1.0*(point1 - point0)
    b1 = point2 - point1
    b2 = point3 - point2

    b0xb1 = np.cross(b0, b1)
    b1xb2 = np.cross(b2, b1)

    b0xb1_x_b1xb2 = np.cross(b0xb1, b1xb2)

    y = np.vdot(b0xb1_x_b1xb2, b1)*(1.0/np.linalg.norm(b1))
    x = np.vdot(b0xb1, b1xb2)

    dihedral = np.degrees(np.arctan2(y, x))

    return dihedral

In [ ]:
boresch_theta_a_lst = []
boresch_theta_b_lst = []
boresch_phi_a_lst = []
boresch_phi_b_lst = []
boresch_phi_c_lst = []
cba_list = []
ABC_list = []

frame_idx = 0
frame_stride = 0

for frame in traj[::frame_stride]:
    c = traj.xyz[frame_idx][c_idx]
    b = traj.xyz[frame_idx][b_idx]
    a = traj.xyz[frame_idx][a_idx]
    A = traj.xyz[frame_idx][A_idx]
    B = traj.xyz[frame_idx][B_idx]
    C = traj.xyz[frame_idx][C_idx]

    cba_angle = angle_calc(c,b,a)
    ABC_angle = angle_calc(A,B,C)
    boresch_theta_a = angle_calc(b,a,A)
    boresch_theta_b = angle_calc(a,A,B)
    boresch_phi_a = dihedral_calc(c,b,a,A)
    boresch_phi_b = dihedral_calc(b,a,A,B)
    boresch_phi_c = dihedral_calc(a,A,B,C)

    boresch_theta_a_lst.append(boresch_theta_a)
    boresch_theta_b_lst.append(boresch_theta_b)
    boresch_phi_a_lst.append(boresch_phi_a)
    boresch_phi_b_lst.append(boresch_phi_b)
    boresch_phi_c_lst.append(boresch_phi_c)
    cba_list.append(cba_angle)
    ABC_list.append(ABC_angle)

    print(frame_idx)
    frame_idx +=frame_stride

In [ ]:
fig,ax = plt.subplots()

bins = np.arange(-180, 180, 0.5)

plt.hist(cba_list, bins=bins, alpha=0.5, density=True, color='blue', linewidth=2.0, label='c-b-a')
plt.hist(ABC_list, bins=bins, alpha=0.5, density=True, color='seagreen', linewidth=2.0, label='A-B-C')
plt.hist(boresch_theta_a_lst, bins=bins, density=True, alpha=0.5, color='hotpink', linewidth=2.0, label='${\\theta_{A}}$')
plt.hist(boresch_theta_b_lst, bins=bins, density=True, alpha=0.5, color='goldenrod', linewidth=2.0, label='${\\theta_{B}}$')
plt.hist(boresch_phi_a_lst, bins=bins, density=True, alpha=0.5, color='indigo', linewidth=2.0, label='${\\phi_{A}}$')
plt.hist(boresch_phi_b_lst, bins=bins, density=True, alpha=0.5, color='tomato', linewidth=2.0, label='${\\phi_{B}}$')
plt.hist(boresch_phi_c_lst, bins=bins, density=True, alpha=0.5, color='brown', linewidth=2.0, label='${\\phi_{C}}$')
plt.xlabel('Angle / $^\circ$', fontsize=35)
plt.ylabel('Probability Density', fontsize=35)

plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xticks([-180,-120,-60,0,60,120,180])
ax.set_xlim([-180.0, 180.0])
ax.legend(fontsize=25, loc='upper right',bbox_to_anchor=(1.26, 1))

plt.gcf().set_size_inches(12,6)

plt.show()